In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, AutoTokenizer
import auxiliary as aux
from tqdm import tqdm
import traceback

c:\Users\samue\Desktop\nlp-project\cubo_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, AutoTokenizer
from auxiliary import ensembler, json_to_Dataset_ensemble
from tqdm import tqdm

class KingBert(nn.Module):
    def __init__(self, distilbert_tuned, albert_tuned):
        super().__init__()
        self.distilbert = distilbert_tuned
        self.albert = albert_tuned

        for distilbert_param in self.distilbert.parameters():
            distilbert_param.requires_grad = False

        for albert_param in self.albert.parameters():
            albert_param.requires_grad = False 
        
        self.alpha = nn.Parameter(0.5 * torch.ones(47), requires_grad=True)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, distilbert_input_ids, albert_input_ids, distil_attention_mask, alb_attention_mask, distilbert_word_ids, albert_word_ids):
        distilbert_output = self.distilbert(input_ids=distilbert_input_ids, attention_mask=distil_attention_mask)
        albert_output = self.albert(input_ids=albert_input_ids, attention_mask=alb_attention_mask)
        distilbert_fixed, albert_fixed = aux.ensembler(distilbert_output['logits'].squeeze(), albert_output['logits'].squeeze(), distilbert_word_ids.squeeze(), albert_word_ids.squeeze())

        distilbert_fixed = self.softmax(distilbert_fixed)
        albert_fixed = self.softmax(albert_fixed)

        final_output = distilbert_fixed * self.alpha + albert_fixed * (torch.ones(47) - self.alpha)

        return self.softmax(final_output)

train_dataset = aux.json_to_Dataset_ensemble('datasets/ensemble_train.json')

distilbert_tuned = AutoModelForTokenClassification.from_pretrained('models/distilbert1')
albert_tuned = AutoModelForTokenClassification.from_pretrained('models/albert1')

kingbert_model = KingBert(distilbert_tuned, albert_tuned)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(kingbert_model.parameters(), lr=2e-5)


num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    for i in tqdm(range(len(train_dataset)), desc="Steps in epoch"):
        try:

            item = train_dataset[i]
            
            distilbert_input_ids = torch.tensor(item['distilbert_inputids']).unsqueeze(0)
            albert_input_ids = torch.tensor(item['albert_inputids']).unsqueeze(0)
            distil_attention_mask = torch.tensor(item['distilbert_attention_masks']).unsqueeze(0)
            alb_attention_mask = torch.tensor(item['albert_attention_masks']).unsqueeze(0)
            distilbert_word_ids = torch.tensor([-100] + item['distilbert_wordids'][1:-1] + [-100]).unsqueeze(0)
            albert_word_ids = torch.tensor([-100] + item['albert_wordids'][1:-1] + [-100]).unsqueeze(0)
            targets = torch.tensor(item['spacy_labels']).unsqueeze(0)
            
            optimizer.zero_grad()
            
            output = kingbert_model(distilbert_input_ids, albert_input_ids, distil_attention_mask, alb_attention_mask, distilbert_word_ids, albert_word_ids)
            
            ohe_targets = torch.zeros(output.shape[0], output.shape[1])
            for i,j in enumerate(targets):
                ohe_targets[i][j] = 1

            loss = criterion(output, ohe_targets)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        except:
            continue
        
    avg_loss = total_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

torch.save(kingbert_model.state_dict(), 'model_state.pth')

print('Training complete.')


Steps in epoch:   0%|          | 0/18651 [00:00<?, ?it/s]c:\Users\samue\Desktop\nlp-project\auxiliary.py:259: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  stacked_tensors1 = torch.stack([torch.tensor(i) for i in output1])
c:\Users\samue\Desktop\nlp-project\auxiliary.py:262: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  stacked_tensors2 = torch.stack([torch.tensor(i) for i in output2])
Steps in epoch:   3%|▎         | 539/18651 [00:49<27:32, 10.96it/s]

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForTokenClassification
from tqdm import tqdm
import auxiliary as aux

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset loading
train_dataset = aux.json_to_Dataset_ensemble('datasets/ensemble_train.json')

# Load fine-tuned base models
distilbert_tuned = AutoModelForTokenClassification.from_pretrained('models/distilbert1').to(device)
albert_tuned = AutoModelForTokenClassification.from_pretrained('models/albert1').to(device)

# Ensembler model
class KingBert(nn.Module):
    def __init__(self, distilbert_tuned, albert_tuned):
        super().__init__()
        self.distilbert = distilbert_tuned
        self.albert = albert_tuned

        for p in self.distilbert.parameters():
            p.requires_grad = False
        for p in self.albert.parameters():
            p.requires_grad = False

        self.alpha = nn.Parameter(0.5 * torch.ones(47, device=device), requires_grad=True)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, distilbert_input_ids, albert_input_ids,
                distil_attention_mask, alb_attention_mask,
                distilbert_word_ids, albert_word_ids):

        distilbert_output = self.distilbert(input_ids=distilbert_input_ids, attention_mask=distil_attention_mask)
        albert_output = self.albert(input_ids=albert_input_ids, attention_mask=alb_attention_mask)

        distilbert_logits = distilbert_output['logits'].squeeze()
        albert_logits = albert_output['logits'].squeeze()
        distilbert_word_ids = distilbert_word_ids.squeeze()
        albert_word_ids = albert_word_ids.squeeze()

        distilbert_fixed, albert_fixed = aux.ensembler(
            distilbert_logits, albert_logits,
            distilbert_word_ids, albert_word_ids
        )

        distilbert_fixed = distilbert_fixed.to(self.alpha.device)
        albert_fixed = albert_fixed.to(self.alpha.device)

        distilbert_fixed = self.softmax(distilbert_fixed)
        albert_fixed = self.softmax(albert_fixed)

        alpha = self.alpha
        one_minus_alpha = (1.0 - alpha).to(alpha.device)  # ensure same device

        combined = distilbert_fixed * alpha + albert_fixed * one_minus_alpha
        return self.softmax(combined)

# Instantiate model
kingbert_model = KingBert(distilbert_tuned, albert_tuned).to(device)

# Optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([kingbert_model.alpha], lr=2e-5)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0
    kingbert_model.train()

    for i in tqdm(range(len(train_dataset)), desc=f"Epoch {epoch+1:02d}"):
        try:
            item = train_dataset[i]

            distilbert_input_ids = torch.tensor(item['distilbert_inputids']).unsqueeze(0).to(device)
            albert_input_ids = torch.tensor(item['albert_inputids']).unsqueeze(0).to(device)
            distil_attention_mask = torch.tensor(item['distilbert_attention_masks']).unsqueeze(0).to(device)
            alb_attention_mask = torch.tensor(item['albert_attention_masks']).unsqueeze(0).to(device)
            distilbert_word_ids = torch.tensor([-100] + item['distilbert_wordids'][1:-1] + [-100]).unsqueeze(0).to(device)
            albert_word_ids = torch.tensor([-100] + item['albert_wordids'][1:-1] + [-100]).unsqueeze(0).to(device)
            labels = torch.tensor(item['spacy_labels']).to(device)

            optimizer.zero_grad()
            output = kingbert_model(distilbert_input_ids, albert_input_ids,
                                    distil_attention_mask, alb_attention_mask,
                                    distilbert_word_ids, albert_word_ids)

            if output.size(0) != labels.size(0):
                continue  # skip if shape mismatch due to token alignment

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        except Exception as e:
            print(f"Skipped example {i} due to error: {e}")
            continue

    avg_loss = total_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Save model
torch.save(kingbert_model.state_dict(), 'model_state.pth')
print('Training complete.')


Using device: cuda


Epoch 01:  11%|█         | 2012/18651 [01:29<12:43, 21.80it/s]

Skipped example 2007 due to error: The size of tensor a (70) must match the size of tensor b (69) at non-singleton dimension 0


Epoch 01:  34%|███▎      | 6266/18651 [04:47<11:07, 18.57it/s]

Skipped example 6263 due to error: The size of tensor a (262) must match the size of tensor b (250) at non-singleton dimension 0


...

Skipped example 15364 due to error: The size of tensor a (98) must match the size of tensor b (97) at non-singleton dimension 0


Epoch 20:  94%|█████████▍| 17592/18651 [14:02<00:50, 20.98it/s]

Skipped example 17588 due to error: The size of tensor a (274) must match the size of tensor b (273) at non-singleton dimension 0


Epoch 20: 100%|██████████| 18651/18651 [14:52<00:00, 20.90it/s]


Epoch [20/20], Loss: 3.0582
Training complete.


In [4]:
model = KingBert(distilbert_tuned=distilbert_tuned, albert_tuned=albert_tuned)
state_dict = torch.load('model_state.pth')
model.load_state_dict(state_dict)
model.eval()

C:\Users\samue\AppData\Local\Temp\ipykernel_14972\3251951415.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('model_state.pth')


KingBert(
  (distilbert): DistilBertForTokenClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): DistilBertSdpaAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
from seqeval.metrics import classification_report as seqeval_classification_report
import torch
from tqdm import tqdm
import numpy as np

def evaluate_ensemble_model(model, dataset, device="cuda", label_pad_token_id=-100):
    model.eval()
    model.to(device)

    all_preds = []
    all_labels = []

    for i in tqdm(range(len(dataset)), desc="Evaluating"):
        try:
            item = dataset[i]

            # Convert and move to device
            d_ids  = torch.tensor(item["distilbert_inputids"]).unsqueeze(0).to(device)
            a_ids  = torch.tensor(item["albert_inputids"]).unsqueeze(0).to(device)
            d_msk  = torch.tensor(item["distilbert_attention_masks"]).unsqueeze(0).to(device)
            a_msk  = torch.tensor(item["albert_attention_masks"]).unsqueeze(0).to(device)
            d_wids = torch.tensor([-100] + item["distilbert_wordids"][1:-1] + [-100]).unsqueeze(0).to(device)
            a_wids = torch.tensor([-100] + item["albert_wordids"][1:-1] + [-100]).unsqueeze(0).to(device)
            labels = torch.tensor(item["spacy_labels"]).unsqueeze(0).to(device)

            # Run forward pass
            with torch.no_grad():
                log_probs = model(d_ids, a_ids, d_msk, a_msk, d_wids, a_wids)  # (1, W, C)

            preds = log_probs.argmax(-1)  # (1, W)
            preds = preds.squeeze(0)
            labels = labels.squeeze(0)

            # Truncate to match shortest sequence
            min_len = min(len(preds), len(labels))
            preds = preds[:min_len]
            labels = labels[:min_len]

            all_preds.append(preds.cpu().numpy().tolist())
            all_labels.append(labels.cpu().numpy().tolist())

        except Exception as e:
            print(f"Skipped example {i} due to error: {e}")
            continue

    # ---- Clean and prepare for evaluation ----
    flat_preds = [p for sublist in all_preds for p in sublist if p != label_pad_token_id]
    flat_labels = [l for sublist in all_labels for l in sublist if l != label_pad_token_id]

    print("Classification report:")
    print(classification_report(flat_labels, flat_preds, zero_division=0))

    cm = confusion_matrix(flat_labels, flat_preds)
    return {
        "precision": np.mean(np.diag(cm) / (cm.sum(0) + 1e-10)),
        "recall": np.mean(np.diag(cm) / (cm.sum(1) + 1e-10)),
        "f1": 2 * np.mean(np.diag(cm) / (cm.sum(1) + 1e-10)) * np.mean(np.diag(cm) / (cm.sum(0) + 1e-10)) / (
            np.mean(np.diag(cm) / (cm.sum(1) + 1e-10)) + np.mean(np.diag(cm) / (cm.sum(0) + 1e-10)) + 1e-10
        ),
        "accuracy": np.mean(np.array(flat_preds) == np.array(flat_labels)),
        "confusion_matrix": cm
    }


In [13]:
test_dataset = aux.json_to_Dataset_ensemble('datasets/ensemble_test.json')


res = evaluate_ensemble_model(model, test_dataset)
print(res)


Evaluating:   0%|          | 0/2320 [00:00<?, ?it/s]c:\Users\samue\Desktop\nlp-project\auxiliary.py:259: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  stacked_tensors1 = torch.stack([torch.tensor(i) for i in output1])
c:\Users\samue\Desktop\nlp-project\auxiliary.py:262: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  stacked_tensors2 = torch.stack([torch.tensor(i) for i in output2])
Evaluating:  45%|████▌     | 1048/2320 [00:48<01:06, 19.24it/s]

Skipped example 1043 due to error: The size of tensor a (363) must match the size of tensor b (275) at non-singleton dimension 0


Evaluating:  86%|████████▋ | 2002/2320 [01:30<00:17, 18.47it/s]

Skipped example 1997 due to error: The size of tensor a (365) must match the size of tensor b (267) at non-singleton dimension 0


Evaluating: 100%|██████████| 2320/2320 [01:45<00:00, 22.05it/s]


Classification report:
              precision    recall  f1-score   support

           0       0.79      0.73      0.76       533
           1       0.69      0.59      0.64       536
           2       0.72      0.80      0.76       772
           3       0.74      0.22      0.34       446
           4       0.63      0.72      0.67       347
           5       0.64      0.53      0.58      1021
           6       0.44      0.44      0.44       837
           7       0.48      0.80      0.60       654
           8       0.65      0.68      0.67       374
           9       0.55      0.74      0.63       771
          10       0.58      0.46      0.51       887
          11       0.83      0.39      0.53       548
          12       0.58      0.59      0.58       429
          13       0.81      0.77      0.79       204
          14       0.46      0.41      0.44       759
          15       0.73      0.83      0.78      1631
          16       0.00      0.00      0.00         5
    